In [1]:
## import libarary 
import os
import pandas as pd
import numpy as np

#import google cloud library
from google.cloud import bigquery
from google.cloud import storage
from google.cloud import aiplatform

## sklearn module
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import joblib

In [2]:
# set up authentication using services account 
# Authenticate using service account
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/home/kevinadityaikhsan15/kevin-025/sa-development.json"

In [3]:
# New data to single predict
new_data = pd.DataFrame({
    'Tenure': [12.0, 1],
    'WarehouseToHome': [26, 5],
    'NumberOfDeviceRegistered': [3, 4],
    'PreferedOrderCat': ['Fashion', 'Mobile Phone'],
    'SatisfactionScore': [4, 5],
    'MaritalStatus': ['Married', 'Single'],
    'NumberOfAddress': [3, 2],
    'Complain': [1, 0],
    'DaySinceLastOrder': [8, 10],
    'CashbackAmount': [155.22, 140.15]
})

new_data

,Tenure,WarehouseToHome,NumberOfDeviceRegistered,PreferedOrderCat,SatisfactionScore,MaritalStatus,NumberOfAddress,Complain,DaySinceLastOrder,CashbackAmount
0,12.0,26,3,Fashion,4,Married,3,1,8,155.22
1,1.0,5,4,Mobile Phone,5,Single,2,0,10,140.15


In [4]:
# Set the project
project_id = 'dti-ds'
dataset_id = 'kevin_dataset_025'
table_id = 'test_table_1'
region = 'us-central1'
bucket_name = 'kevin_gcs_025'
blob_name = 'data/test_table_1.csv'
model_name = 'xgb_model.pkl'

In [5]:
# Retrieve the model from Google Cloud Storage
try:
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket(bucket_name)
    blob_model = bucket.blob(f'model/{model_name}')
    blob_model.download_to_filename('xgb_model.pkl')

    print("Read Model Succeeded")
except:
    raise TypeError("An exception Occured")

Read Model Succeeded


In [6]:
# Load data from BQ

# Using bigquery client 
client = bigquery.Client(project=project_id)

# Load unseen data from BiQuery 
query_job = client.query(f"""select * from {dataset_id}.{table_id}""")
df = query_job.result().to_dataframe()
df

/home/kevinadityaikhsan15/miniconda3/envs/model-env/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,Tenure,WarehouseToHome,NumberOfDeviceRegistered,PreferedOrderCat,SatisfactionScore,MaritalStatus,NumberOfAddress,Complain,DaySinceLastOrder,CashbackAmount
0,0.0,8.0,4,Mobile Phone,1,Single,8,0,0.0,127.85
1,0.0,12.0,3,Mobile Phone,5,Single,2,0,0.0,125.52
2,0.0,14.0,4,Mobile Phone,5,Single,2,1,0.0,126.00
3,0.0,30.0,1,Mobile Phone,5,Single,2,0,0.0,110.91
4,0.0,14.0,5,Mobile Phone,5,Single,1,0,0.0,123.25
...,...,...,...,...,...,...,...,...,...,...
645,1.0,10.0,4,Mobile Phone,1,Single,3,1,1.0,140.69
646,1.0,18.0,5,Mobile Phone,5,Single,3,1,1.0,151.00
647,1.0,7.0,6,Mobile Phone,5,Single,2,0,1.0,148.25
648,1.0,25.0,2,Mobile Phone,5,Married,2,0,1.0,154.56


In [7]:
# Load the model from the file
loaded_model = joblib.load('xgb_model.pkl') 

# Now you can use loaded_model to make predictions
predictions = loaded_model.predict(new_data)
predictions

array([0, 1])

In [8]:
# Create new df
new_test_unseen = new_data.copy()
new_test_unseen['Prediction'] = predictions
new_test_unseen

,Tenure,WarehouseToHome,NumberOfDeviceRegistered,PreferedOrderCat,SatisfactionScore,MaritalStatus,NumberOfAddress,Complain,DaySinceLastOrder,CashbackAmount,Prediction
0,12.0,26,3,Fashion,4,Married,3,1,8,155.22,0
1,1.0,5,4,Mobile Phone,5,Single,2,0,10,140.15,1


In [9]:
# Bulk Prediction to unseen data
bulk_prediction = loaded_model.predict(df)
bulk_probabilities = loaded_model.predict_proba(df)[:, 1]
df['Prediction'] = bulk_prediction
df['Probabilities'] = bulk_probabilities
df

,Tenure,WarehouseToHome,NumberOfDeviceRegistered,PreferedOrderCat,SatisfactionScore,MaritalStatus,NumberOfAddress,Complain,DaySinceLastOrder,CashbackAmount,Prediction,Probabilities
0,0.0,8.0,4,Mobile Phone,1,Single,8,0,0.0,127.85,1,0.976885
1,0.0,12.0,3,Mobile Phone,5,Single,2,0,0.0,125.52,0,0.053828
2,0.0,14.0,4,Mobile Phone,5,Single,2,1,0.0,126.00,1,0.999451
3,0.0,30.0,1,Mobile Phone,5,Single,2,0,0.0,110.91,1,0.845634
4,0.0,14.0,5,Mobile Phone,5,Single,1,0,0.0,123.25,1,0.995530
...,...,...,...,...,...,...,...,...,...,...,...,...
645,1.0,10.0,4,Mobile Phone,1,Single,3,1,1.0,140.69,1,0.979651
646,1.0,18.0,5,Mobile Phone,5,Single,3,1,1.0,151.00,1,0.995076
647,1.0,7.0,6,Mobile Phone,5,Single,2,0,1.0,148.25,1,0.941694
648,1.0,25.0,2,Mobile Phone,5,Married,2,0,1.0,154.56,1,0.942250


In [10]:
# Upload to BigQuery Dataset as a Table
table_id = 'test_data_predicted'
client = bigquery.Client()

# Define the Full Table ID
table_id_full = f"{client.project}.{dataset_id}.{table_id}"

# Define new schema fields to add
schema = [
    bigquery.SchemaField("Tenure", "FLOAT"),
    bigquery.SchemaField("WarehouseToHome", "FLOAT"),
    bigquery.SchemaField("NumberOfDeviceRegistered", "INTEGER"),
    bigquery.SchemaField("PreferedOrderCat", "STRING"),
    bigquery.SchemaField("SatisfactionScore", "INTEGER"),
    bigquery.SchemaField("MaritalStatus", "STRING"),
    bigquery.SchemaField("NumberOfAddress", "INTEGER"),
    bigquery.SchemaField("Complain", "INTEGER"),
    bigquery.SchemaField("DaySinceLastOrder", "FLOAT"),
    bigquery.SchemaField("CashbackAmount", "FLOAT"),
    bigquery.SchemaField("Prediction", "INTEGER"),
    bigquery.SchemaField("Probabilities", "FLOAT")
]

In [11]:
# Load DataFrame into the BigQuery table
job = client.load_table_from_dataframe(df, table_id_full)

# Wait for the job to complete
job.result()
print(f"Loaded {job.output_rows} rows into {table_id_full}")

Loaded 650 rows into dti-ds.kevin_dataset_025.test_data_predicted
